# ETL Process
---
 In this file, it will 
 * read data inside 'fall2024data/'
 * convert data into one dataframe
 * change the name of features
 * drop rows which containing Nan or Inf value
 * save plot of features to 'Featrues_plot/'
 * save Analystic data to 'Analysis/'
 * save processed data to csv in 'Datasets/'

    'Dataset.csv'           - whole dataset of traffic\
    'BENIGN.csv'            - set of data labeled 'BENIGN'\
    'DoS_GoldenEye.csv'     - set ofdata labeled 'DoS_GoldenEye'\
    'DoS_Hulk.csv'          - set of data labeled 'DoS_Hulk'\
    'DoS_Slowhttptest.csv'  - set of data labeled 'DoS_Slowttptest'
    
 ...

## Extraction
---
This part will load data from the folder and concatenate them into one DataFrame


In [1]:
import numpy as np
import pandas as pd
import pyarrow.parquet as pq
import glob
import time
from functools import wraps

In [2]:
def logged(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start_time = time.time()
        start_time_formatted = time.strftime('%H:%M:%S', time.localtime(start_time))
        print(f"[{start_time_formatted}] Function '{func.__name__}' start.")
        
        result = func(*args, **kwargs)
        
        end_time = time.time()
        end_time_formatted = time.strftime('%H:%M:%S', time.localtime(end_time))
        execution_time = end_time - start_time
        print(f"[{end_time_formatted}] Function End, Duration: {execution_time:.4f}Sec")
        
        return result
    return wrapper

In [3]:
@logged
def get_Dset(fpath:str)->pd.DataFrame:
    _ids = list()
    try:
        # get csv files
        for csvfile in glob.glob(f'{fpath}/*.csv'):
            print('{:30s}'.format(csvfile), 'found')
            _ids.append(pd.read_csv(csvfile, sep=','))

        # get json files
        for jsonfile in glob.glob(f'{fpath}/*.json'):
            print('{:30s}'.format(jsonfile), 'found')
            _ids.append(pd.read_json(jsonfile, lines=True))

        # get parquet files
        for pqfile in glob.glob(f'{fpath}/*.parquet'):
            buff = pq.read_table(pqfile)
            print('{:30s}'.format(pqfile), 'found')
            _ids.append(buff.to_pandas())

        return pd.concat(_ids, ignore_index=True)
    
    except Exception as e:
        print('Exception:', e)
        return
    
    

In [4]:
ids = get_Dset('fall2024data')

[23:12:36] Function 'get_Dset' start.
fall2024data/ids_2.csv         found
fall2024data/ids_0.csv         found
fall2024data/ids_1.csv         found
fall2024data/ids_7.json        found
fall2024data/ids_10.json       found
fall2024data/ids_9.json        found
fall2024data/ids_4.json        found
fall2024data/ids_3.json        found
fall2024data/ids_5.parquet     found
fall2024data/ids_6.parquet     found
fall2024data/ids_11.parquet    found
fall2024data/ids_8.parquet     found
[23:12:37] Function End, Duration: 0.6366Sec


In [5]:
ids.shape

(61128, 79)

## Transform
---
In this part, data will be separated by its Label and processed to show some insight
* Data types conversion
* Data format conversion (cm to inches, etc.)
* Remove duplicates
* Identifying errors in data
* Handling out-of-range and outlier data
* Add any other transformations you find necessary.

Also, Drop Label 'Heartbleed'


In [6]:
@logged
def drop_Heartbleed(data:pd.DataFrame):
    # Drop Label 'Heartbleed'
    Hbd = (data.iloc[:,-1] == 'Heartbleed')
    H_idx = Hbd[Hbd == True].index
    print(H_idx.shape[0], 'items dropped')
    df = data.drop(H_idx)
    return df

df = drop_Heartbleed(ids)


[23:12:37] Function 'drop_Heartbleed' start.
11 items dropped
[23:12:37] Function End, Duration: 0.0495Sec


### General Info about data

In [7]:
# 61117 samples with 78 features and 1 label
print(df.shape)

(61117, 79)


In [8]:
#All of features are in numerical type, thus, we don't need to transform it.
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 61117 entries, 0 to 61127
Data columns (total 79 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0    Destination Port             61117 non-null  int64  
 1    Flow Duration                61117 non-null  int64  
 2    Total Fwd Packets            61117 non-null  int64  
 3    Total Backward Packets       61117 non-null  int64  
 4   Total Length of Fwd Packets   61117 non-null  int64  
 5    Total Length of Bwd Packets  61117 non-null  int64  
 6    Fwd Packet Length Max        61117 non-null  int64  
 7    Fwd Packet Length Min        61117 non-null  int64  
 8    Fwd Packet Length Mean       61117 non-null  float64
 9    Fwd Packet Length Std        61117 non-null  float64
 10  Bwd Packet Length Max         61117 non-null  int64  
 11   Bwd Packet Length Min        61117 non-null  int64  
 12   Bwd Packet Length Mean       61117 non-null  float64
 13   Bwd P

In [9]:
print(pd.unique(df.iloc[:,-1]))

['BENIGN' 'DoS Hulk' 'DoS Slowhttptest' 'DoS GoldenEye']


### Change the name of the features

In [10]:
@logged
def strip_cols(df:pd.DataFrame)->pd.DataFrame:
    # Some of features have confusing spaces in their name
    cols = df.columns.to_list()
    
    for i in range(len(cols)):
        print('{:30} ->'.format(cols[i]), end= ' ')
        cols[i] = cols[i].strip()
        print('{:30}'.format(cols[i]))
        
    return df.set_axis(cols, axis=1)
        

In [11]:
df_stripped = strip_cols(df)

[23:12:37] Function 'strip_cols' start.
 Destination Port              -> Destination Port              
 Flow Duration                 -> Flow Duration                 
 Total Fwd Packets             -> Total Fwd Packets             
 Total Backward Packets        -> Total Backward Packets        
Total Length of Fwd Packets    -> Total Length of Fwd Packets   
 Total Length of Bwd Packets   -> Total Length of Bwd Packets   
 Fwd Packet Length Max         -> Fwd Packet Length Max         
 Fwd Packet Length Min         -> Fwd Packet Length Min         
 Fwd Packet Length Mean        -> Fwd Packet Length Mean        
 Fwd Packet Length Std         -> Fwd Packet Length Std         
Bwd Packet Length Max          -> Bwd Packet Length Max         
 Bwd Packet Length Min         -> Bwd Packet Length Min         
 Bwd Packet Length Mean        -> Bwd Packet Length Mean        
 Bwd Packet Length Std         -> Bwd Packet Length Std         
Flow Bytes/s                   -> Flow Bytes/s    

In [12]:
df_stripped.columns

Index(['Destination Port', 'Flow Duration', 'Total Fwd Packets',
       'Total Backward Packets', 'Total Length of Fwd Packets',
       'Total Length of Bwd Packets', 'Fwd Packet Length Max',
       'Fwd Packet Length Min', 'Fwd Packet Length Mean',
       'Fwd Packet Length Std', 'Bwd Packet Length Max',
       'Bwd Packet Length Min', 'Bwd Packet Length Mean',
       'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s',
       'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min',
       'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max',
       'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std',
       'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags',
       'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length',
       'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s',
       'Min Packet Length', 'Max Packet Length', 'Packet Length Mean',
       'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count',
       'SYN Flag Co

### Drop rows which contains Nan or Inf value

In [13]:
@logged
def drop_anomaly(df:pd.DataFrame)->pd.DataFrame:
    # Convert Inf value into Nan
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    
    # Columns containing NaN value
    print(df.isna().sum().sum(), 'items dropped')
    
    # 'DoS Hulk' and 'BENIGN' contains Nan or Inf value
    print(np.unique(df.loc[(ids.count(axis=1) < df.shape[1]), :].to_numpy()[:,-1], return_counts=True))
    return df.dropna()


In [14]:
df_dropped = drop_anomaly(df_stripped)

[23:12:37] Function 'drop_anomaly' start.
232 items dropped
(array(['DoS Hulk'], dtype=object), array([114]))
[23:12:37] Function End, Duration: 0.0916Sec


In [15]:
df_dropped.isna().sum().sum()

0

In [16]:
df_dropped.shape

(61001, 79)

In [17]:
BENIGN = df_dropped.loc[df_dropped['Label'] == 'BENIGN']
DoS_GoldenEye = df_dropped.loc[df_dropped['Label'] == 'DoS GoldenEye']
DoS_Hulk = df_dropped.loc[df_dropped['Label'] == 'DoS Hulk']
DoS_Slowhttptest = df_dropped.loc[df_dropped['Label'] == 'DoS Slowhttptest']

### Plotting each features
Just so watch distribution

In [18]:
import matplotlib.pyplot as plt

@logged
def show_fig(target:list):
    for j in range(len(BENIGN.columns)-1):
        fig, ax = plt.subplots(len(target), 1, constrained_layout=True)
        fig.set_dpi(600)
    
        target_col = j
        fig.suptitle(BENIGN.columns[target_col])
    
        for i in range(len(target)):
            
            ax[i].set_title(target[i].iloc[0,-1])
            ax[i].scatter(range(target[i].shape[0]), 
                          target[i].iloc[:,target_col].to_numpy(),
                          marker='x', 
                          s=[5 for _ in range(target[i].shape[0])])
        
        fig.savefig(f"Features_plot/{j}_{BENIGN.columns[j].replace('/', '')}.jpeg", dpi=600)
        plt.close(fig)
        print(f"Features_plot/{j}_{BENIGN.columns[j].replace('/', '')}.jpeg")

In [19]:
show_fig([BENIGN, DoS_GoldenEye, DoS_Hulk, DoS_Slowhttptest])

[23:12:37] Function 'show_fig' start.
Features_plot/0_Destination Port.jpeg
Features_plot/1_Flow Duration.jpeg
Features_plot/2_Total Fwd Packets.jpeg
Features_plot/3_Total Backward Packets.jpeg
Features_plot/4_Total Length of Fwd Packets.jpeg
Features_plot/5_Total Length of Bwd Packets.jpeg
Features_plot/6_Fwd Packet Length Max.jpeg
Features_plot/7_Fwd Packet Length Min.jpeg
Features_plot/8_Fwd Packet Length Mean.jpeg
Features_plot/9_Fwd Packet Length Std.jpeg
Features_plot/10_Bwd Packet Length Max.jpeg
Features_plot/11_Bwd Packet Length Min.jpeg
Features_plot/12_Bwd Packet Length Mean.jpeg
Features_plot/13_Bwd Packet Length Std.jpeg
Features_plot/14_Flow Bytess.jpeg
Features_plot/15_Flow Packetss.jpeg
Features_plot/16_Flow IAT Mean.jpeg
Features_plot/17_Flow IAT Std.jpeg
Features_plot/18_Flow IAT Max.jpeg
Features_plot/19_Flow IAT Min.jpeg
Features_plot/20_Fwd IAT Total.jpeg
Features_plot/21_Fwd IAT Mean.jpeg
Features_plot/22_Fwd IAT Std.jpeg
Features_plot/23_Fwd IAT Max.jpeg
Features

In [20]:
df_dropped.describe().to_csv('Analysis/ids_describe.csv')

In [21]:
DoS_GoldenEye.describe().to_csv('Analysis/GoldenEye_describe.csv')
DoS_Hulk.describe().to_csv('Analysis/Hulk_describe.csv')
DoS_Slowhttptest.describe().to_csv('Analysis/Slowhttptest_describe.csv')

# Load
---


In [22]:
DoS_Slowhttptest.to_csv('Datasets/DoS_Slowhttptest.csv')
DoS_Hulk.to_csv('Datasets/DoS_Hulk.csv')
DoS_GoldenEye.to_csv('Datasets/DoS_GoldenEye.csv')

In [24]:
df_dropped.to_csv('Datasets/Dataset.csv')